# Angle Mechanisms

In this notebook, we will lay out our approach for a Neural Network approach to predict a pose based on landmark. Additionally, we are storing out angle calculation and perspectively our angle error calculation.

In [1]:
import numpy as np
import pandas as pd

## Getting the Data

### Image upload and Preprocessing, single picture

In [186]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os
import pandas as pd

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display


In [187]:
# Model Initialization
model_name = "movenet_lightning"
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
model = module.signatures['serving_default']

In [188]:
# Choose image file and preprocess

image_path = "../clean/00000134.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


In [191]:
# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
xyz_individual = outputs["output_0"].numpy()


### Drawing the Landmarks

In [192]:
# Helper method for drawing landmarks
def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

### Uploading whole image folder 

In [193]:
# Get list of image file names in given directory
dir_path_train_dd = "../clean"
file_names = []
for entry in os.listdir(dir_path_train_dd):
    file_names.append(entry)

In [194]:
# Create DF of xyz values
df = []
image_names = []

for file in file_names:
    image_names.append(dir_path_train_dd)
    dir_path_train_dd = f"../clean/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
#       print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        df.append(outputs["output_0"].numpy().reshape(51).tolist())


In [195]:
data = pd.DataFrame(df)

In [196]:
data["label"] = "downdog"

In [198]:
data.head(3)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,label
0,0.614686,0.366069,0.722570,0.612122,0.350986,0.643456,0.612995,0.350938,0.715975,0.563880,...,0.515043,0.705209,0.887617,0.685450,0.883067,0.797601,0.667293,0.864593,0.652384,downdog
1,0.534696,0.722514,0.564541,0.529799,0.733964,0.405342,0.526650,0.732507,0.399033,0.498358,...,0.471535,0.375810,0.738871,0.586280,0.311128,0.580621,0.628086,0.273356,0.892581,downdog
2,0.569520,0.529188,0.600391,0.572470,0.513902,0.517100,0.575602,0.515343,0.508116,0.547930,...,0.502984,0.773414,0.708213,0.674067,0.870783,0.614912,0.665915,0.863403,0.627864,downdog


In [200]:
# Create DF of xyz values
df = []
xyz = []
for file in file_names:
    dir_path_train_dd = f"../clean/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
#         print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        df.append(outputs["output_0"].numpy().reshape(51).tolist())
        xyz.append(outputs["output_0"].numpy()[0][0])

## Angle Calculator

In [203]:
# Getting specific image out of xyz-coordinate file and transforming it to 2D (renamed to l for simplicity)

image_landmarks = xyz[6][:, :-1]
lm = image_landmarks
lm

array([[0.66960835, 0.35614344],
       [0.67016554, 0.33723804],
       [0.6718497 , 0.33988258],
       [0.6360858 , 0.30929214],
       [0.6374579 , 0.31444016],
       [0.58631724, 0.3324954 ],
       [0.58074266, 0.331631  ],
       [0.70875835, 0.27349722],
       [0.6967046 , 0.26761028],
       [0.7763366 , 0.1773637 ],
       [0.75954664, 0.19102556],
       [0.40240896, 0.53494716],
       [0.40276787, 0.5312333 ],
       [0.56703186, 0.63760805],
       [0.57365   , 0.6361871 ],
       [0.7234893 , 0.7441256 ],
       [0.7093338 , 0.7298353 ]], dtype=float32)

In [204]:
# Used Keypoints in data (corresponding to row number)

KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [25]:
# Function for getting angles for single image input,
# returns dict with angle name as key and list with only the values

def angle_function(lm):
    
    landmark_dict = {
    'landmarks_left_elbow' : (lm[5], lm[7], lm[9]),
    'landmarks_right_elbow' : (lm[6], lm[8], lm[10]),
    'landmarks_left_shoulder' : (lm[7], lm[5], lm[11]),
    'landmarks_right_shoulder' : (lm[8], lm[6], lm[12]),
    'landmarks_hip_left' : (lm[5], lm[11], lm[13]),
    'landmarks_hip_right' : (lm[6], lm[12], lm[14]),
    'landmarks_left_knee' : (lm[11], lm[13], lm[15]),
    'landmarks_right_knee' : (lm[12], lm[14], lm[16])
        }
    
    def angle_calculator(landmarks):

        # Converting the points into numpy arrays
        p1 = np.array(landmarks[0])
        p2 = np.array(landmarks[1])
        p3 = np.array(landmarks[2])

        # Creating the Vectors between two points
        vec_p1 = p1-p2
        vec_p2 = p3-p2

        # Calculating the cosine of the angle
        cosine_angle = np.dot(vec_p1,vec_p2) / (np.linalg.norm(vec_p1)*np.linalg.norm(vec_p2))
        
        #clipping cosine angle
        cosine_angle = np.clip(cosine_angle, -1, 1)
        # Calculating angle
        angle = np.arccos(cosine_angle)

        # Calculating degrees
        angle_degrees = np.degrees(angle)

        return angle_degrees

    angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

    angle_dict = {}
    
    for dictkey, listkey in zip(landmark_dict, angle_name_list):
        i = angle_calculator(landmark_dict[dictkey])
        angle_dict[listkey] = i
    
    angle_list = [angle_dict[key] for key in angle_dict]
    
    return angle_dict, angle_list

## Angle Comparison Tool

In [209]:
# This loop creates a list of lists with the angle value for each picture respectively

lst = []
for x in xyz:
    lst.append(angle_function(x[:, :-1])[1])   

# Dataframe for the list
df = pd.DataFrame(lst)

In [210]:
# Angle name list for the column names of the dataframe
angle_name_list = ['angle_elbow_left',
                   'angle_elbow_right',
                   'angle_shoulder_left',
                   'angle_shoulder_right',
                   'angle_hip_left',
                   'angle_hip_right',
                   'angle_knee_left',
                   'angle_knee_right',
                  ]

# Adding the created column names
df.columns = angle_name_list

In [211]:
# Getting image (path) names to name the indices
index_image_names = pd.Index(image_names)[1:]

# set index
df.set_index(index_image_names, inplace=True)

In [215]:
df.head(5)

,angle_elbow_left,angle_elbow_right,angle_shoulder_left,angle_shoulder_right,angle_hip_left,angle_hip_right,angle_knee_left,angle_knee_right
../clean/00000372.jpg,161.533622,167.843977,176.212090,179.972024,64.985133,65.501503,155.154463,154.631616
../clean/00000414.jpg,156.647111,161.688982,172.806634,170.628852,76.427716,76.186424,178.631264,177.908468
../clean/00000158.jpg,163.160640,168.443158,163.976623,173.624648,83.535028,79.592601,174.322075,172.935975
../clean/00000164.jpg,159.131123,162.453301,175.071327,176.938227,67.050041,66.100435,170.020585,173.788581
../clean/00000170.jpg,161.461897,159.272757,178.728352,179.944047,63.241979,62.176024,172.617380,170.075123


In [48]:
# df.to_csv('/Users/lennartjanssen/code/lennijanssen/angles_test')

## Angle Error & Score

In [241]:
#Finding values for the best pose

# Choose image file and preprocess

image_path = "../clean/00000144.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
best_pose = outputs["output_0"].numpy()[:, :, :, :-1][0][0]

In [242]:
#Finding values for the test pose

# Choose image file and preprocess

image_path = "../clean/00000338.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
test_pose = outputs["output_0"].numpy()[:, :, :, :-1][0][0]

In [243]:
test_pose

array([[0.5708826 , 0.40645045],
       [0.576467  , 0.39673132],
       [0.57615376, 0.38526136],
       [0.51849204, 0.3919344 ],
       [0.52068865, 0.35362864],
       [0.45254654, 0.4695372 ],
       [0.44141775, 0.32939914],
       [0.6168701 , 0.40841112],
       [0.58001006, 0.25417495],
       [0.789742  , 0.3339388 ],
       [0.7096631 , 0.14340983],
       [0.24649948, 0.7028052 ],
       [0.24908094, 0.61588585],
       [0.4586088 , 0.7828223 ],
       [0.47202104, 0.7173096 ],
       [0.6309516 , 0.89589274],
       [0.6396793 , 0.81330293]], dtype=float32)

In [244]:
from operator import truediv

def angle_comparer(test, best):

    # get angles for each pose
    best_pose_angles = angle_function(best)[1]
    test_pose_angles = angle_function(test)[1]
    
    # find differences between each by substraction
    angle_substraction = [a - b for a, b in zip(best_pose_angles, test_pose_angles)]
    
    # change values to absolute values
    angle_difference = list(map(abs, angle_substraction))
    
    # find percentages for each pose
    test_angle_percentage = list(map(truediv, test_pose_angles, best_pose_angles))
    
    # find absolute difference by percentage
    test_angle_percentage_diff = [round(abs(x-1),2) for x in test_angle_percentage]
    
    # find average percentage difference
    average_percentage_diff = round(sum(test_angle_percentage_diff)/len(test_angle_percentage_diff),2)
    
    
    return test_angle_percentage_diff, average_percentage_diff
    

In [245]:
angle_comparer(test_pose,best_pose)

([0.01, 0.02, 0.13, 0.14, 0.05, 0.13, 0.06, 0.03], 0.07)